In [3]:
import pylxd
import os

from pylxd import Client

## Authenticating

In [4]:
BASE_DIR = os.getcwd()
def cert_path(file):
    return os.path.join(BASE_DIR, 'test_certificate', 'tmp', file)

## Before executing this need to make sure that
core.trust_password is set to expected password

```
sudo lxc config get core.trust_password
```

```
sudo lxc config set core.trust_password test
```

# or use following command to get trusted token

```
lxc config trust add
```



In [5]:
cert = (cert_path('lxd.crt'),cert_path('lxd.key'))
endpoint = 'https://192.168.1.230:8443'

trusted = 'eyJjbGllbnRfbmFtZSI6InRlc3QtZGV2IiwiZmluZ2VycHJpbnQiOiIyZGI4OGZiMGNjYTdiYTM1ZjI0ZjcyNjAzMTQxYjdhOGUzNDAzODBiMGNmZmJmZmY2MzRjN2E5Y2VjZjNlNGEyIiwiYWRkcmVzc2VzIjpbIjE5Mi4xNjguMS4yMzA6ODQ0MyJdLCJzZWNyZXQiOiJjMDE2MDIzMWUwYjIxYTYxMzVhYmZjYWZmMjU3Y2Y2MmEyNTJjNGFiMTE4MDhmY2I1YjMxZTI5MDU0ZDE3MzhjIiwiZXhwaXJlc19hdCI6IjAwMDEtMDEtMDFUMDA6MDA6MDBaIiwidHlwZSI6IiJ9'

client: pylxd.Client = pylxd.Client(endpoint=endpoint, cert=cert, verify=False)

client.authenticate(trusted)
print(client.trusted)

True


In [21]:

allMembersInCluster = client.cluster.members.all()

for member in allMembersInCluster:
    print(f"{member.server_name}: {member.groups} {member.status}")


/mnt/c/Users/asus/Desktop/Projects/cloudboi/backend/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/mnt/c/Users/asus/Desktop/Projects/cloudboi/backend/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/mnt/c/Users/asus/Desktop/Projects/cloudboi/backend/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthe

cloudboi-main: ['default'] Online
subvm0: ['default', 'cloudboi-resource'] Offline
subvm1: ['default', 'cloudboi-resource'] Online


In [7]:
container = client.instances.get("u3")

command = ["passwd", "root"]
stdin_payload = bytearray("ThisMe1234\nThisMe1234\n", "utf-8")

exit_code, stdout, stderr = container.execute(command, stdin_payload=stdin_payload, encoding="utf-8")

print("Exit Code:", exit_code)
print("STDOUT:", stdout)
print("STDERR:", stderr)
print("Container Status:", container.state().status)


/mnt/c/Users/asus/Desktop/Projects/cloudboi/backend/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/mnt/c/Users/asus/Desktop/Projects/cloudboi/backend/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/mnt/c/Users/asus/Desktop/Projects/cloudboi/backend/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthe

Exit Code: 0
STDOUT: 
You can now choose the new password or passphrase.

A valid password should be a mix of upper and lower case letters, digits, and
other characters.  You can use a password containing at least 7 characters
from all of these classes, or a password containing at least 8 characters
from just 3 of these 4 classes.
An upper case letter that begins the password and a digit that ends it do not
count towards the number of character classes used.

A passphrase should be of at least 3 words, 11 to 72 characters long, and
contain enough different characters.

Alternatively, if no one else can see your terminal now, you can pick this as
your password: "Serum-baltic+Relish".


STDERR: passwd: updating all authentication tokens for user root.
Enter new password: Re-type new password: passwd: all authentication tokens updated successfully.

Container Status: Running


In [25]:
import asyncio
import websockets
import ssl
import nest_asyncio

nest_asyncio.apply()

container = client.instances.get("u3")

res = container.raw_interactive_execute(['/bin/bash'])

ws_url = f"wss://192.168.1.230:8443{res['ws']}"

ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Function to handle the WebSocket communication
async def connect_websocket():
    try:
        async with websockets.connect(ws_url, ssl=ssl_context, ping_interval=30, ping_timeout=20) as websocket:
            print("Connected to WebSocket server!")
            
            # Send initial message if needed
            # await websocket.send("Hello, server!")
            # initial_response = await websocket.recv()
            # print(f"Initial response: {initial_response}")
            
            return websocket
    except Exception as e:
        print(f"Connection error: {e}")
        return None

async def send_message(websocket, message):
    if websocket:
        try:
            # Add a small delay after connecting
            await asyncio.sleep(0.5)
            
            await websocket.send(message)
            
            # Add timeout to recv
            response = await asyncio.wait_for(websocket.recv(), timeout=10)
            return response
        except asyncio.TimeoutError:
            print("Timeout waiting for response")
            return None
        except Exception as e:
            print(f"Error sending/receiving message: {e}")
            return None

# Function to close the connection
async def close_connection(websocket):
    if websocket:
        try:
            await websocket.close()
            print("Connection closed")
        except Exception as e:
            print(f"Error closing connection: {e}")

# For Jupyter use - create a connection and store it in a variable
async def create_connection():
    return await connect_websocket()

async def debug_websocket():
    try:
        async with websockets.connect(ws_url, ssl=ssl_context) as websocket:
            print(f"Connected to {ws_url}")
            
            # Print connection details
            print(f"Local address: {websocket.local_address}")
            print(f"Remote address: {websocket.remote_address}")
            
            # Try a simple ping
            pong = await websocket.ping()
            await asyncio.wait_for(pong, timeout=5)
            print("Ping successful")
            
            # Try with a simple message
            await websocket.send("test")
            print("Message sent, waiting for response...")
            
            response = await asyncio.wait_for(websocket.recv(), timeout=10)
            print(f"Response received: {response}")
            
    except Exception as e:
        print(f"Debug error: {e}")

# Run it
asyncio.get_event_loop().run_until_complete(debug_websocket())

# Run this cell to establish the connection
# websocket_connection = asyncio.get_event_loop().run_until_complete(create_connection())

# Example usage in subsequent cells:
# response = asyncio.get_event_loop().run_until_complete(send_message(websocket_connection, "Your message"))
# print(response)

# When finished:
# asyncio.get_event_loop().run_until_complete(close_connection(websocket_connection))

/mnt/c/Users/asus/Desktop/Projects/cloudboi/backend/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/mnt/c/Users/asus/Desktop/Projects/cloudboi/backend/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Connected to wss://192.168.1.230:8443/1.0/operations/fa7f5903-7698-4b97-878d-4e9c06e5ef85/websocket?secret=5e4e58a2ef8165ff3462a077dcaffbcaf18f676f38cf99d0578b765168a80500
Local address: ('172.31.78.244', 53888)
Remote address: ('192.168.1.230', 8443)
Ping successful
Message sent, waiting for response...
Debug error: no close frame received or sent


In [6]:
import asyncio
import websockets
import ssl
import json
import nest_asyncio
import warnings
from urllib3.exceptions import InsecureRequestWarning

# Suppress SSL warnings
warnings.simplefilter('ignore', InsecureRequestWarning)

# Apply nest_asyncio for Jupyter
nest_asyncio.apply()

# SSL context
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Get fresh connection details
async def get_connection_urls(container_name):
    container = client.instances.get(container_name)
    res = container.raw_interactive_execute(['/bin/bash'])
    
    # Extract both WebSocket URLs
    main_ws_url = f"wss://192.168.1.230:8443{res['ws']}"
    control_ws_url = f"wss://192.168.1.230:8443{res['control']}"
    
    print(f"Main WebSocket URL: {main_ws_url}")
    print(f"Control WebSocket URL: {control_ws_url}")
    
    return main_ws_url, control_ws_url

# Connect to both WebSockets
async def connect_lxd_terminal(container_name):
    main_url, control_url = await get_connection_urls(container_name)
    
    # Connect to both WebSockets
    main_ws = await websockets.connect(main_url, ssl=ssl_context, ping_interval=None)
    control_ws = await websockets.connect(control_url, ssl=ssl_context, ping_interval=None)
    
    print("Connected to LXD WebSockets")
    
    # Initialize shell via control socket
    # control_msg = json.dumps({
    #     "command": "shell",
    #     "arguments": {
    #         "width": 80,
    #         "height": 24
    #     }
    # })
    
    # await control_ws.send(control_msg)
    # print("Shell initialized via control socket")
    
    # Wait for initial greeting from the shell
    try:
        initial_output = await asyncio.wait_for(main_ws.recv(), timeout=2)
        if isinstance(initial_output, bytes):
            initial_text = initial_output.decode('utf-8', errors='replace')
            print(f"Initial shell output(Bytes): {initial_text}")
        else:
            print(f"Initial shell output: {initial_output}")
    except asyncio.TimeoutError:
        print("No initial output from shell (may be normal)")
    
    # Clear any remaining initial messages
    await clear_pending_messages(main_ws)
    
    return main_ws, control_ws

# Function to clear any pending messages
async def clear_pending_messages(websocket, timeout=0.5):
    pending_messages = []
    try:
        while True:
            message = await asyncio.wait_for(websocket.recv(), timeout=timeout)
            if isinstance(message, bytes):
                message = message.decode('utf-8', errors='replace')
            pending_messages.append(message)
    except asyncio.TimeoutError:
        if pending_messages:
            print(f"Cleared {len(pending_messages)} pending messages")
            for msg in pending_messages:
                print(f"Cleared: {msg}")
    return pending_messages

# Function to execute a command with improved output collection
async def execute_command(main_ws, command, timeout=10):
    if not command.endswith('\n'):
        command += '\n'
    
    # Clear any pending messages before sending new command
    await clear_pending_messages(main_ws)
    
    # Send command through main WebSocket
    await main_ws.send(command)
    print(f"Sent command: {command.strip()}")
    
    # Wait a moment for command to be processed
    await asyncio.sleep(0.5)
    
    # Collect all output for a fixed time
    output_chunks = []
    start_time = asyncio.get_event_loop().time()
    
    while asyncio.get_event_loop().time() - start_time < timeout:
        try:
            chunk = await asyncio.wait_for(main_ws.recv(), timeout=1)
            
            if isinstance(chunk, bytes):
                text = chunk.decode('utf-8', errors='replace')
            else:
                text = chunk
            
            output_chunks.append(text)
            print(f"DEBUG - Received chunk: {repr(text)}")
            
            # Stop collecting if we've been idle for a bit
            if len(output_chunks) > 0 and asyncio.get_event_loop().time() - start_time > 2:
                break
                
        except asyncio.TimeoutError:
            # If we got some output and now there's a timeout, we're probably done
            if output_chunks:
                break
    
    # Combine all output pieces
    full_output = ''.join(output_chunks)
    
    # Remove the command echo from the beginning if present
    command_echo = command.strip() + '\r\n'
    if full_output.startswith(command_echo):
        full_output = full_output[len(command_echo):]
    
    # Remove prompt if present at the end
    prompt_patterns = ['\r\n[root@', '\n[root@']
    for pattern in prompt_patterns:
        if pattern in full_output:
            full_output = full_output.split(pattern)[0]
    
    print(f"DEBUG - Collected {len(output_chunks)} chunks, total length: {len(full_output)}")
    
    return full_output

# Close connections
async def close_connections(main_ws, control_ws):
    # Send exit command
    await main_ws.send("exit\n")
    
    # Close both WebSockets
    await asyncio.sleep(1)  # Give it time to process
    await main_ws.close()
    await control_ws.close()
    print("WebSocket connections closed")

# For Jupyter notebook usage
async def jupyter_terminal_connect(container_name):
    return await connect_lxd_terminal(container_name)

async def jupyter_terminal_command(main_ws, command):
    return await execute_command(main_ws, command)

async def jupyter_terminal_close(main_ws, control_ws):
    await close_connections(main_ws, control_ws)

In [11]:
# First cell - connect
container_name = "u3"  # Your container name
main_ws, control_ws = await jupyter_terminal_connect(container_name)

# Second cell - run a command
output = await jupyter_terminal_command(main_ws, "ls -la")
print("Command output:")
print(output)

# Third cell - run more commands
output = await jupyter_terminal_command(main_ws, "whoami")
print(output)

# When finished
await jupyter_terminal_close(main_ws, control_ws)

Main WebSocket URL: wss://192.168.1.230:8443/1.0/operations/e702694e-8b97-46c4-af0b-042a34924978/websocket?secret=37755caf624650df45a9983bf1a500884c2fcd837b5ed174a44ccb6331517f85
Control WebSocket URL: wss://192.168.1.230:8443/1.0/operations/e702694e-8b97-46c4-af0b-042a34924978/websocket?secret=516f831adcf6cb538c9f5f8bf4b88cab25dc380944194ab43785130f5e31a2bb
Connected to LXD WebSockets
Initial shell output(Bytes): [root@u3 ~]# 
Sent command: ls -la
DEBUG - Collected 0 chunks, total length: 0
Command output:

Sent command: whoami
DEBUG - Collected 0 chunks, total length: 0

WebSocket connections closed


In [7]:
main_ws, control_ws = await jupyter_terminal_connect("u3")

Main WebSocket URL: wss://192.168.1.230:8443/1.0/operations/147c32e4-ecd1-481e-a232-87a4593b350c/websocket?secret=35696a7c9fd1938878960c3ddd4d62fbc419507f8b310b00f4439b13aba4444f
Control WebSocket URL: wss://192.168.1.230:8443/1.0/operations/147c32e4-ecd1-481e-a232-87a4593b350c/websocket?secret=43b18c4e751c0e255a262af98b391fb1c9ef0c09b144e4a10d48913c2284bd81
Connected to LXD WebSockets
Initial shell output(Bytes): [root@u3 ~]# 


In [10]:
await main_ws.send("ls -la\n")
output = await asyncio.wait_for(main_ws.recv(), timeout=10)
# Check if it time out
print(output)

TimeoutError: 

In [ ]:

def get_instance(instance_name):  
  try:
    instance = client.instances.get(instance_name)
    return instance
  except pylxd.exceptions.LXDAPIException as e:
    print(e)
    return None
  
test_instance = get_instance('test-instance')

Instance not found


In [ ]:
all_image = client.images.all()

for image in all_image:
    print(image.alias)
    print(image.fingerprint)

In [19]:
def create_instance(instance_name):
  instance_info = {
      'name': instance_name,
      "source": {
            "type": "image",
            "mode": "pull",
            "server": "https://cloud-images.ubuntu.com/releases",
            "protocol": "simplestreams",
            "alias": "22.04",
        },
      'profiles': ['default'],
      'config': {
        'limits.cpu': '1',
        'limits.memory': '256MB'
      }
    }
  try:
    instance = client.instances.create(instance_info, wait=True, target='@cloudboi-resource')
    return instance
  except pylxd.exceptions.LXDAPIException as e:
    print(e)
    return
  
test_instance = create_instance('test-instance')

client.instances.get('test-instance')

/mnt/c/Users/asus/Desktop/Projects/cloudboi/backend/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/mnt/c/Users/asus/Desktop/Projects/cloudboi/backend/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/mnt/c/Users/asus/Desktop/Projects/cloudboi/backend/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthe

Failed creating instance record: Instance "test-instance" already exists


/mnt/c/Users/asus/Desktop/Projects/cloudboi/backend/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.230'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
def start_instance(instance_name):
  try:
    instance = get_instance(instance_name)
    instance.start(wait=False)
    return instance
  except pylxd.exceptions.LXDAPIException as e:
    print(e)
    return
  
print(start_instance('test-instance'))

In [ ]:
def delete_instance(instance_name):
  try:
    instance = client.instances.get(instance_name)
    instance.delete(wait=True)
  except pylxd.exceptions.LXDAPIException as e:
    print(e)
    return
# Cant delete instance if it is running
delete_instance('test-instance')

Instance is running


In [ ]:
# execute command 
def execute_command(instance_name, command):
  try:
    instance = get_instance(instance_name)
    result = instance.execute(command)
    return result
  except pylxd.exceptions.LXDAPIException as e:
    print(e)
    return
  
print(execute_command('test-instance', ['ls', '-l']))

Traceback (most recent call last):
  File "/Users/pannapat/.vscode/extensions/ms-python.python-2024.20.0-darwin-arm64/python_files/python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 11, in <module>
  File "<string>", line 5, in execute_command
  File "/Users/pannapat/Desktop/cloudboi/backend/.venv/lib/python3.12/site-packages/pylxd/models/instance.py", line 475, in execute
    stdin.connect()
  File "/Users/pannapat/Desktop/cloudboi/backend/.venv/lib/python3.12/site-packages/ws4py/client/__init__.py", line 214, in connect
    self.sock = ssl.wrap_socket(self.sock, **self.ssl_options)
                ^^^^^^^^^^^^^^^
AttributeError: module 'ssl' has no attribute 'wrap_socket'

